## Reinforcement learning - TME 3 - Q-learning

L'objectif du TME est d'implémenter les algorithmes de renforcement value-based étudiés en cours (Q-learning et ses variantes) et de les tester dans un framework classique (gym de open-ai, MDP GridWorld).

In [1]:
import matplotlib
#from matplotlib import pyplot as plt

matplotlib.use("TkAgg")
import gym
import gridworld
from gym import wrappers, logger
import numpy as np
import copy
from scipy.sparse import dok_matrix, lil_matrix

### Implémentation des algorithmes 

Trois agents sont implémentés :
+ RandomAgent : un agent aléatoire
+ PolicyIteration : un agent qui implémente l'algorithme Policy Iteration
+ ValueIteration : un agent qui implémente l'algorithme Value Iteration

Pour Policy Iteration et Value Iteration, on code une première implémentation naïve qui utilise la structure de données sous forme de dictionnaire. Cette implémentation naïve fonctionne en temps raisonnable pour tous les exemples donnés sauf l'exemple 9 (qui possède environ 200 000 états). Pour gérer cet exemple, on fait une implémentation plus sophistiquée utilisant des matrices sparse scipy.

In [2]:
env = gym.make("gridworld-v0")
env.setPlan("gridworldPlans/plan1.txt", {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1})
statedic, mdp = env.getMDP()

In [3]:
class RandomAgent(object):
    def __init__(self, env, n_actions, alpha, gamma, tau=1):
        self.n_actions = n_actions
        
    def act(self, observation, reward, done):
        return np.random.randint(0, self.n_actions)

In [16]:
class ValueIterationAgent(object):
    """Agent implementing Value Iteration. Naive implementation with dictionary structure."""

    def __init__(self, env, n_actions, alpha, gamma, tau=1):
        self.env = env
        self.action_space = env.action_space
        self.statedic, self.mdp = env.getMDP()
        self.policy = {}
        for state, state_id in self.statedic.items():
            if state in self.mdp:
                list_actions = self.mdp[state].keys()
                self.policy[state_id] = self.action_space.sample()

    def act(self, observation, reward, done):
        try:
            return self.policy[self.statedic[self.env.state2str(observation)]]
        except:
            return 0
                
    def train(self, eps=5e-4, gamma=0.99):  # Value Iteration algorithm
        value = {}
        for state, state_id in self.statedic.items():
            value[state_id] = 0
            
        distance = np.inf
        while distance > eps:
            new_value = {}
            
            for state, state_id in self.statedic.items():
                if state in self.mdp:
                    results = [sum([proba*(reward + gamma*value[self.statedic[new_state]]) for (proba, new_state, reward, done) in transitions]) for action, transitions in self.mdp[state].items()]
                    new_value[state_id] = np.max(results)
                else:
                    new_value[state_id] = value[state_id]
                    
            distance = np.linalg.norm(np.array(list(value.values()))-np.array(list(new_value.values())), ord=np.inf)
            value = new_value
                    
        for state, state_id in self.statedic.items():
                if state in self.mdp:
                    results = [sum([proba*(reward + gamma*value[self.statedic[new_state]]) for (proba, new_state, reward, done) in transitions]) for action, transitions in self.mdp[state].items()]
                    self.policy[state_id] = np.argmax(results)

In [5]:
class Q_learning(object):
    """Implementing a Q learning agent with Boltzmann selection"""

    def __init__(self, env, n_actions, alpha, gamma, tau=1):
        self.env = env
        self.Q = np.zeros((100, n_actions))
        self.n_actions = n_actions
        self.action = 0
        self.state = 0
        self.alpha = alpha
        self.gamma = gamma
        self.tau = tau
        self.dic_state = {}
        self.nS = 0
        self.first = True
        
    def act(self, observation, reward, done):
        if self.first:
            self.first = False
            return np.random.choice(range(self.n_actions))
        
        if self.env.state2str(observation) in self.dic_state:
            new_state = self.dic_state[self.env.state2str(observation)]
        else:
            new_state = len(self.dic_state)
            self.dic_state[self.env.state2str(observation)] = len(self.dic_state)
            
            
        if not done:
            self.Q[self.state][self.action] = self.Q[self.state][self.action] + self.alpha * (reward + self.gamma * np.max(self.Q[new_state]) - self.Q[self.state][self.action])
        else:
            self.Q[self.state][self.action] = self.Q[self.state][self.action] + self.alpha * (reward - self.Q[self.state][self.action])
        
        self.state = new_state
        # boltzmann
        probas = np.exp(self.Q[self.state] / self.tau)
        probas = probas / np.sum(probas)
        self.action = np.random.choice(range(self.n_actions), p=probas)
        
        return self.action
        

In [6]:
class DynaQ_learning(object):
    """Implementing a Dyna-Q learning agent with Boltzmann selection"""

    def __init__(self, env, n_actions, alpha, gamma, tau=1):
        self.env = env
        self.Q = np.zeros((11, n_actions))
        self.R = np.zeros((11, n_actions, 11))
        self.P = np.zeros((11, n_actions, 11))
        self.n_actions = n_actions
        self.action = 0
        self.state = 0
        self.alpha = alpha
        self.gamma = gamma
        self.tau = tau
        self.dic_state = {}
        self.nS = 0
        self.first = True
        self.done = False
        self.c = 0
        
    def act(self, observation, reward, done):
   #     if self.first:
   #         self.first = False
   #         return np.random.choice(range(self.n_actions))
        
    
        if self.env.state2str(observation) in self.dic_state:
            new_state = self.dic_state[self.env.state2str(observation)]
        else:
            new_state = len(self.dic_state)
            self.dic_state[self.env.state2str(observation)] = len(self.dic_state)
             
        if new_state == 0 and reward > 0:
            print(self.state)
            print(done)
            print(self.env.state2str(observation))
            raise ValueError('non')
            
        if not self.done:
            if not done:
                self.Q[self.state][self.action] = self.Q[self.state][self.action] + self.alpha * (reward + self.gamma * np.max(self.Q[new_state]) - self.Q[self.state][self.action])
            if done:
                self.Q[self.state][self.action] = self.Q[self.state][self.action] + self.alpha * (reward - self.Q[self.state][self.action])

            alphaR = 0.2
            
            self.R[self.state][self.action][new_state] = (1-alphaR)*self.R[self.state][self.action][new_state] + alphaR*reward
            self.P[self.state][self.action] = (1-alphaR)*self.P[self.state][self.action]
            self.P[self.state][self.action][new_state] = self.P[self.state][self.action][new_state] + alphaR

            k = 10
            for i in range(k):
                s = np.random.randint(0, len(self.dic_state))
                a = np.random.randint(0, self.n_actions)
                for s2 in range(len(self.dic_state)):
                    if self.R[s][a][s2] + self.gamma * np.max(self.Q[s2]) > 1:
                        print(s, a, s2)
                        for k,v in self.dic_state.items():
                            if v == s2:
                                print('destiniation')
                                print(k)
                            if v == s:
                                print('origine')
                                print(k)
                        print(self.R[s][a][s2])
                        print(self.R)
                        print(np.max(self.Q[s2]))
                        raise ValueError('oh oh')
                self.Q[s][a] = (1 - self.alpha)*self.Q[s][a] + self.alpha*np.sum([self.P[s][a][s2]*(self.R[s][a][s2] + self.gamma * np.max(self.Q[s2])) for s2 in range(len(self.dic_state))])

        self.state = new_state
            
        self.done = done
        # boltzmann
        try:
            probas = np.exp(self.Q[self.state] / self.tau)
            probas = probas / np.sum(probas)
            self.action = np.random.choice(range(self.n_actions), p=probas)
        except:
            print(self.Q[self.state])
            raise ValueError('ot')
        
        
        return self.action
        

In [7]:
class SARSA(object):
    """Implementing a SARSA agent with Boltzmann selection"""

    def __init__(self, env, n_actions, alpha, gamma, tau=1):
        self.env = env
        self.Q = np.zeros((100, n_actions))
        self.n_actions = n_actions
        self.action = 0
        self.state = 0
        self.alpha = alpha
        self.gamma = gamma
        self.tau = tau
        self.dic_state = {}
        self.nS = 0
        self.first = True
        self.epochs = 1
        
    def act(self, observation, reward, done):
     #   if self.first:
     #       self.first = False
     #       return np.random.choice(range(self.n_actions))
        
        if self.env.state2str(observation) in self.dic_state:
            new_state = self.dic_state[self.env.state2str(observation)]
        else:
            
            new_state = len(self.dic_state)
            self.dic_state[self.env.state2str(observation)] = len(self.dic_state)
            
            
        # boltzmann
       # probas = np.exp(self.Q[self.state] / self.tau)
       # probas = probas / np.sum(probas)
        # eps greedy
        eps = 0.1 / np.sqrt(self.epochs)
        if np.random.random_sample() < eps:
            new_action = np.random.choice(range(self.n_actions))
            if self.epochs > 4000:
                print('exploration choice')
        else:
            new_action = np.argmax(self.Q[new_state])
            
        if not done:
            self.Q[self.state][self.action] = self.Q[self.state][self.action] + self.alpha * (reward + self.gamma * self.Q[new_state][new_action] - self.Q[self.state][self.action])
        else:
            self.epochs += 1
            self.Q[self.state][self.action] = self.Q[self.state][self.action] + self.alpha * (reward - self.Q[self.state][self.action])
        
        self.state = new_state
        self.action = new_action
        
        return self.action
        

### Test sur la plateforme gym - problème GridWorld 

On commence par encapsuler le protocole de test dans des fonctions. Ces fonctions techniques ne présentent pas d'intérêt algorithmique.

In [8]:
def create_env(env_number, rewards):
    env = gym.make("gridworld-v0")
    env.setPlan("gridworldPlans/plan" + str(env_number) + ".txt", rewards)
    env.seed(0)  # Initialise le seed du pseudo-random
            
    return env

In [ ]:
outdir = 'gridworld-v0/agent-results'
envm = wrappers.Monitor(env, directory=outdir, force=True, video_callable=False)
for i in range(n_runs):
    obs = envm.reset()
    reward = 0 
    done = False 
    env.render(0.1)
    while True:
        action = agent.act(obs, reward, done)
        if done:
            break
        obs, reward, done, _ = envm.step(action) 
        env.render()


In [9]:
def run_experiment(env, agent, n_runs=10, verbose=1):
    outdir = 'gridworld-v0/agent-results'
    envm = wrappers.Monitor(env, directory=outdir, force=True, video_callable=False)
    reward = 0
    done = False
    rsum = 0
    rtot = []
    for i in range(n_runs):
        obs = envm.reset()
        reward = 0 ####
        done = False ####
        if verbose > 1:
            env.render(0.1)
        j = 0
        rsum = 0
        while True:
            action = agent.act(obs, reward, done)
            if done:
                if verbose > 1:
                    print("Episode : " + str(i) + " rsum=" + str(rsum) + ", " + str(j) + " actions")
                break
            obs, reward, done, _ = envm.step(action) #####
            rsum += reward
            j += 1
            if verbose > 1:
                env.render()
            
        rtot.append(rsum)

    if verbose:
        print("Mean reward for agent %s over %d runs: %.2f" % (agent.__class__.__name__, n_runs, np.mean(np.array(rtot))))
    env.close()
    return rtot

In [10]:
def test_agent(env_number, rewards, alpha, gamma, tau, AgentClass=Q_learning, run_exp=True, n_runs=10, verbose=3):
    env = create_env(env_number, rewards)
    agent = AgentClass(env, env.nA, alpha, gamma, tau)
    if AgentClass == ValueIterationAgent:
        agent.train()
    if verbose:
        print('Test - Env %d - %s - %d runs' % (env_number, AgentClass.__name__, n_runs))
    if verbose > 2:
        print('Policy of ' + agent.__class__.__name__ +  ': ' +str(agent.policy))
    if run_exp:
        rtot = run_experiment(env, agent, n_runs, verbose)
    if verbose:
        print(' ')
    return rtot, agent

On peut comparer le reward moyen de l'agent optimal à celui de l'agent aléatoire. Comme prévu, l'agent optimal fait mieux que l'agent aléatoire.

In [11]:
rewards = {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1}

In [17]:
alphas = [0.01, 0.05, 0.1, 0.2, 0.5, 1]
taus = [0.005, 0.01, 0.05, 0.1, 0.2]
results = np.zeros((len(alphas), len(taus)))
for i in range(len(alphas)):
    for j in range(len(taus)):
        results[i][j] = np.mean(np.array([test_agent(1, rewards, alphas[i], 0.99, taus[j], AgentClass=Q_learning, run_exp=True, n_runs=1000, verbose=0)[0]]*5))

KeyboardInterrupt: 

In [81]:
alphas = [0.01, 0.05, 0.1, 0.2, 0.5, 1]
taus = [0.005, 0.01, 0.05, 0.1, 0.2]
results_SARSA = np.zeros((len(alphas), len(taus)))
for i in range(len(alphas)):
    for j in range(len(taus)):
        results_SARSA[i][j] = np.mean(np.array([test_agent(1, rewards, alphas[i], 0.99, taus[j], AgentClass=SARSA, run_exp=True, n_runs=1000, verbose=0)[0]]*5))

In [49]:
results_SARSA

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [45]:
results

array([[1.693355, 1.726217, 1.289149, 1.258777, 1.085849],
       [1.936201, 1.421532, 1.334497, 1.348095, 1.348891],
       [1.557488, 1.630154, 1.443283, 1.376981, 1.335862],
       [1.825666, 1.562489, 1.558177, 1.586178, 1.10239 ],
       [1.951264, 1.954646, 1.959491, 1.946639, 1.569964],
       [1.857181, 1.854962, 1.867367, 1.858754, 1.869576]])

In [50]:
results

array([[ 0.676402,  0.658348,  0.185805,  0.014817,  0.013005],
       [ 0.590284,  0.565209,  0.406066,  0.191175,  0.157219],
       [ 0.503001,  0.513095,  0.446625,  0.240555,  0.067447],
       [ 0.515446,  0.513229,  0.375796,  0.257763,  0.099999],
       [ 0.444595,  0.319243,  0.408213,  0.266893,  0.165366],
       [-0.1436  , -0.113914, -0.098177, -0.175775, -0.082476]])

In [17]:
r_best, a_best = test_agent(0, rewards, 0.1, 0.99, 0.01, AgentClass=ValueIterationAgent, run_exp=True, n_runs=1000, verbose=1)

Test - Env 0 - ValueIterationAgent - 1000 runs
Mean reward for agent ValueIterationAgent over 1000 runs: 0.98
 


In [13]:
r_random = test_agent(0, rewards, 0.1, 0.99, 0.1, AgentClass=RandomAgent, run_exp=True, n_runs=100, verbose=1)

Test - Env 0 - RandomAgent - 100 runs
Mean reward for agent RandomAgent over 100 runs: -0.73
 


In [15]:
r_q, a_q = test_agent(0, rewards, 0.5, 0.99, 0.005, AgentClass=Q_learning, run_exp=True, n_runs=1000, verbose=1)

Test - Env 0 - Q_learning - 1000 runs
Mean reward for agent Q_learning over 1000 runs: 0.96
 


In [18]:
r_dynaq, a_dynaq = test_agent(0, rewards, 0.5, 0.99, 0.01, AgentClass=DynaQ_learning, run_exp=True, n_runs=1000, verbose=1)

Test - Env 0 - DynaQ_learning - 1000 runs
Mean reward for agent DynaQ_learning over 1000 runs: 0.95
 


In [20]:
a_dynaq.Q

array([[ 0.84323308, -0.04      ,  0.4632109 ,  0.39177339],
       [ 0.58754653,  0.90400915,  0.68841289,  0.7524059 ],
       [ 0.69708818,  0.31907755,  0.56699352,  0.87366154],
       [ 0.51448633,  0.36278006,  0.43051372,  0.77413447],
       [ 0.54447938,  0.        ,  0.40658246,  0.41437358],
       [ 0.42872979,  0.32722888,  0.95040964, -0.04      ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.22578476,  0.24554285,  0.38063976,  0.26410808],
       [ 0.31904207,  0.22355225,  0.31123707,  0.11292418],
       [ 0.32064457,  0.        ,  0.        ,  0.99404439],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [127]:
np.max(a_dynaq.Q)

2.660393640650938

In [122]:
env = gym.make("gridworld-v0")
env.setPlan("gridworldPlans/plan0.txt", {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1})
run_experiment(env, a_dynaq, 100, 1)

Mean reward for agent DynaQ_learning over 100 runs: 0.96


[0.981,
 0.8649999999999999,
 0.969,
 0.986,
 0.972,
 0.964,
 0.991,
 0.946,
 0.969,
 0.974,
 0.944,
 0.978,
 0.967,
 0.97,
 0.969,
 0.966,
 0.9289999999999999,
 0.973,
 0.975,
 0.947,
 0.974,
 0.984,
 0.968,
 0.954,
 0.951,
 0.954,
 0.983,
 0.971,
 0.976,
 0.939,
 0.954,
 0.955,
 0.958,
 0.943,
 0.966,
 0.966,
 0.97,
 0.977,
 0.98,
 0.954,
 0.964,
 0.993,
 0.971,
 0.95,
 0.98,
 0.967,
 0.994,
 0.976,
 0.9129999999999999,
 0.973,
 0.991,
 0.98,
 0.982,
 0.976,
 0.966,
 0.993,
 0.9179999999999999,
 0.948,
 0.973,
 0.982,
 0.946,
 0.9339999999999999,
 0.975,
 0.974,
 0.9309999999999999,
 0.951,
 0.8939999999999999,
 0.968,
 0.949,
 0.962,
 0.9249999999999999,
 0.9259999999999999,
 0.991,
 0.9319999999999999,
 0.958,
 0.99,
 0.946,
 0.99,
 0.98,
 0.9219999999999999,
 0.9089999999999999,
 0.8999999999999999,
 0.971,
 0.9369999999999999,
 0.986,
 0.976,
 0.975,
 0.981,
 0.974,
 0.949,
 0.973,
 0.976,
 0.962,
 0.968,
 0.96,
 0.976,
 0.9319999999999999,
 0.8969999999999999,
 0.834999999999999

In [170]:
r_sarsa, a_sarsa = test_agent(0, rewards, 0.001, 0.99, 0.01, AgentClass=SARSA, run_exp=True, n_runs=2000, verbose=1)

Test - Env 0 - SARSA - 2000 runs
Mean reward for agent SARSA over 2000 runs: 0.91
 


In [172]:
np.max(a_sarsa.Q)

0.814469809467841

In [79]:
_ = run_experiment(env, a_sarsa, 1000, 1)

exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
exploration choice
Mean reward for agent SARSA over 1000 runs: 1.95


In [71]:
dic = {}
for key, value in a_best.statedic.items():
    if key in a_sarsa.dic_state:
        dic[value] = np.argmax(a_sarsa.Q, axis=1)[a_sarsa.dic_state[key]]
print(dic)

{0: 0, 2: 1, 3: 2, 4: 1, 5: 0, 7: 3, 8: 1, 9: 0, 10: 3, 11: 3, 12: 3, 14: 3, 15: 2, 16: 2, 17: 1, 18: 1, 19: 3}


In [39]:
matplotlib.pyplot.plot(range(len(r_sarsa)), np.cumsum(np.array(r_sarsa)))
matplotlib.pyplot.plot(range(len(r_best)), np.cumsum(np.array(r_best)))
matplotlib.pyplot.plot(range(len(r_q)), np.cumsum(np.array(r_q)))
matplotlib.pyplot.show()

KeyboardInterrupt: 

Pour les environnements 4 et 7, un phénomène intéressant se produit : il faut passer par un malus pour arriver à la sortie. Dans ces cas, il faut bien choisir la combinaison de gamma et du reward des cases vides (`reward[0]`) pour que l'agent apprenne. Par exemple, la combinaison `reward[0] = -0.001` et `gamma=0.99` ne fonctionne pas. Les combinaisons `reward[0] = -0.01` et `gamma=0.99` ou `reward[0] = -0.001` et `gamma=1` fonctionnent (mais donnent des comportements plus ou moins prudents dans le cas de l'environnement 7).

In [ ]:
test_agent(4, {0: -0.001, 3: 1, 4: 1, 5: -1, 6: -1}, run_exp=True, n_runs=1, verbose=2, gamma=1)

In [20]:
test_agent(7, {0: -0.01, 3: 1, 4: 1, 5: -1, 6: -1}, run_exp=True, n_runs=5, verbose=2, gamma=0.99)

MDP loaded
Test - Env 7 - ValueIterationAgent - 5 runs
Episode : 0 rsum=2.4699999999999993, 58 actions
Episode : 1 rsum=2.4799999999999995, 57 actions
Episode : 2 rsum=-0.07000000000000006, 9 actions
Episode : 3 rsum=0.8999999999999999, 13 actions
Episode : 4 rsum=0.8799999999999999, 15 actions
Mean reward for agent ValueIterationAgent over 5 runs: 1.33
 


Enfin, pour l'environnement 9, on utilise l'agent OptimizedValueIterationAgent. L'exécution du test nécessite environ 2 minutes et 4 GB de mémoire vive. L'essentiel de ce budget est consacré à la construction du MDP et sa traduction sous la bonne forme de données (matrices sparse). La phase d'entraînement à proprement parler est très rapide (~ 10 secondes).

In [13]:
test_agent(9, {0: -0.01, 3: 1, 4: 1, 5: -1, 6: -1}, AgentClass=OptimizedValueIterationAgent, run_exp=True, n_runs=2, verbose=2, gamma=0.99)

MDP loaded
Test - Env 9 - OptimizedValueIterationAgent - 1 runs
Episode : 0 rsum=3.850000000000006, 122 actions
Mean reward for agent OptimizedValueIterationAgent over 1 runs: 3.85
 
